In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

In [7]:
# Let's test the base model before training
prompt = "Who was the first woman to win a Nobel Prize?"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(
    **inputs,
    max_length=50,
    num_beams=4,
    early_stopping=True,
    top_k=50,
    top_p=0.95,
    repetition_penalty=1.2,
    no_repeat_ngram_size=3,
    length_penalty=1.2,
    temperature=1.0,
    do_sample=True
)

print("Before SFT (chat format prompt):")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

inputs = tokenizer(prompt, return_tensors='pt').to(device)
outputs = model.generate(
    **inputs,
    max_length=50,
    num_beams=4,
    early_stopping=True,
    top_k=50,
    top_p=0.95,
    repetition_penalty=1.2,
    no_repeat_ngram_size=3,
    length_penalty=1.2,
    temperature=1.0,
    do_sample=True
)

print("Before SFT (general format prompt):")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Before SFT (chat format prompt):
user
Who was the first woman to win a Nobel Prize?
What was the name of the woman who won the Nobel Prize in 1902 for her work on the microscope?Mary Anning
What did Mary Anning
Before SFT (general format prompt):
Who was the first woman to win a Nobel Prize?

Nobel Prize in Physiology or Medicine 1903

What was the name of the man who won the Nobel Prize in physiology or medicine for his discovery of penicillin?

